In [23]:
import numpy as np
import cv2 as cv
import glob
import os
    
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0005)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
image_folder = 'E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img'  #   raw_img

images = glob.glob(os.path.join(image_folder, "*.jpg"))


i=0
for fname in images:
    print(fname)
    img_name = fname.split('\\')[-1]
    img_name = img_name.split('.')[0]
    img = cv.imread(fname)
    img = cv.resize(img,(int(img.shape[1]/1.5),int(img.shape[0]/1.5)))
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(img, (9,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        i+=1
        print('Pic %s: there are corner'%i)
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners2, ret)
        print('write image to out_%s'%img_name    )
        cv.imwrite('E:/NUS/5003/Camera_cali/ivy_data/LabCamera/processed_img/out_%s.png'%img_name, img)
        #cv.imshow('img', img)
        #cv.waitKey(500)
        
        #cv.destroyAllWindows()


E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\1.jpg
Pic 1: there are corner
write image to out_1
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\10.jpg
Pic 2: there are corner
write image to out_10
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\11.jpg
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\12.jpg
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\13.jpg
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\14.jpg
Pic 3: there are corner
write image to out_14
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\15.jpg
Pic 4: there are corner
write image to out_15
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\16.jpg
Pic 5: there are corner
write image to out_16
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\17.jpg
Pic 6: there are corner
write image to out_17
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\2.jpg
Pic 7: there are corner
write image to out_2
E:/NUS/5003/A_3Dtransform/ivy_data/LabCamera/raw_img\20.jpg
Pic 8: there are corne

In [24]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
mtx

array([[2.48164984e+03, 0.00000000e+00, 6.88424831e+02],
       [0.00000000e+00, 2.53145860e+03, 9.93322708e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [25]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )  

## result = 0.67  it is good

total error: 0.3375671440278838


In [17]:
img = cv.imread('32.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [18]:
# undistort 
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('32_output.jpg', dst)

True

In [9]:
roi

(448, 73, 940, 352)

In [10]:
mtx

array([[2.48164984e+03, 0.00000000e+00, 6.88424831e+02],
       [0.00000000e+00, 2.53145860e+03, 9.93322708e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [16]:
## write matrix to csv

In [17]:
import csv 
import pandas as pd
mtx_df = pd.DataFrame(mtx)
mtx_df.to_csv('parameter_mtx2.csv')

In [21]:
import numpy as np

In [27]:
mtx1 = np.array([[2.47081936e+03, 0.00000000e+00, 7.17150916e+02],
       [0.00000000e+00, 2.49958908e+03, 1.22085184e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

mtx2 = np.array([[2.48164984e+03, 0.00000000e+00, 6.88424831e+02],
       [0.00000000e+00, 2.53145860e+03, 9.93322708e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

avg_mtx = (mtx1+mtx2)/2
avg_mtx

array([[2.47623460e+03, 0.00000000e+00, 7.02787874e+02],
       [0.00000000e+00, 2.51552384e+03, 1.10708727e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])